# Precipitation Variability Across Timescales

This notebook demonstrates how to use the precipitation variability metrics driver. 

This notebook should be run in an environment with python, jupyterlab, pcmdi metrics package, and cdat installed. It is expected that you have downloaded the sample data as demonstrated in [the download notebook](Demo_0_download_data.ipynb).  

The following cell reads in the choices you made during the download data step:

In [1]:
from user_choices import demo_data_directory, demo_output_directory

## Basic Use

Use the `--help` flag for assistance with the precip variability driver:

In [2]:
%%bash
variability_across_timescales_PS_driver.py --help

usage: variability_across_timescales_PS_driver.py [-h]
                                                  [--parameters PARAMETERS]
                                                  [--diags OTHER_PARAMETERS [OTHER_PARAMETERS ...]]
                                                  [--mip MIP] [--mod MOD]
                                                  [--var VAR] [--frq FRQ]
                                                  [--modpath MODPATH]
                                                  [--results_dir RESULTS_DIR]
                                                  [--case_id CASE_ID]
                                                  [--prd PRD] [--fac FAC]
                                                  [--nperseg NPERSEG]
                                                  [--noverlap NOVERLAP]
                                                  [--cmec] [--no_cmec]

optional arguments:
  -h, --help            show this help message and exit
  --parameters PARAMETERS, -p PARAMETER

Settings are specified in a separate parameter file, which is printed here:

In [3]:
# print parameter file
with open("basic_precip_variability_param.py") as f:
    print(f.read())

mip = "cmip5"
exp = "historical"
mod = "GISS-E2-H"
var = "pr"
frq = "day"
modpath = 'demo_data_standard/CMIP5_demo_precip_var/'
results_dir = 'demo_output/precip_variability/'
prd = [2000,2001]  # analysis period
fac = 86400  # factor to make unit of [mm/day]

# length of segment in power spectra (~10 years)
# shortened to 2 years for demo purposes
nperseg = 2 * 365
# length of overlap between segments in power spectra (~5 years)
# shortened to 1 year for demo purposes
noverlap = 1 * 365



The parameter file is passed to the driver using the `-p` flag, similar to other PMP metrics. The next cell runs the driver as a subprocess:

In [ ]:
%%bash
variability_across_timescales_PS_driver.py -p basic_precip_variability_param.py --cmec

Running the precipitation variability driver produces three output files, found in the demo output directory:
Spatial pattern of spectral power (forced and unforced variability)

In [ ]:
!ls {demo_output_directory + "/precip_variability"}

Metrics results are found in the JSON output.

In [ ]:
import json
import os
output_path = os.path.join(demo_output_directory,"precip_variability/PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.r6i1p1.json")
with open(output_path) as f:
    metric = json.load(f)["RESULTS"]
print(json.dumps(metric, indent=2))

In [ ]:
# Need to add model/obs ratio